In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '3'
import tensorflow as tf
import numpy as np
import PIL
#import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import Sequential
from tensorflow.compat.v1 import ConfigProto

model_path = "/media/2/Network/pretrained_model/vgg_model.h5"
#model_path = "/media/2/Network/pretrained_model/back_layers.h5"
#img_path = "/media/2/Network/Imagenet_dup/val/n02074367"# dugong
data_path = "/media/2/Network/Imagenet_dup/"
feature4_path = "/media/2/Network/extracted_feature/whole_not_shuffle_to_15"

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.4
config.gpu_options.allow_growth = True
np.random.seed(0)
tf.__version__

'2.3.0'

In [4]:
error_feature_path = "/media/2/Network/extracted_feature/whole_not_shuffle_to_15/seq_16_pkt_error"
feature_list =  os.listdir(error_feature_path)
feature_list = sorted(feature_list)
len(feature_list)
# load feature from disk
# classify feature
test_feature = []
test_label = []
train_feature = []
train_label =[]
val_feature = []
val_label =[]
for data in feature_list:
    #print(feature4_path+"/"+data)
    if 'train' in data:
        if 'feature' in data:
            tmp = np.load(error_feature_path+"/"+data,mmap_mode='c')
            train_feature.append(tmp)
        else :
            tmp = np.load(error_feature_path+"/"+data,mmap_mode='c')
            train_label.append(tmp)
       #print("train in",data)
    elif 'test' in data:
        #print("test in",data)
        if 'feature' in data:
            tmp = np.load(error_feature_path+"/"+data,mmap_mode='c')
            test_feature.append(tmp)
        else:
            tmp = np.load(error_feature_path+"/"+data,mmap_mode='c')
            test_label.append(tmp)
    elif 'validation' in data:
        #print("val in",data)
        if 'feature' in data:
            tmp = np.load(error_feature_path+"/"+data,mmap_mode='c')
            val_feature.append(tmp)
        else:
            tmp = np.load(error_feature_path+"/"+data,mmap_mode='c')
            val_label.append(tmp)


In [5]:
model=None
def ready_model(model_path,layer):
    model=load_model(model_path) # whole model
    # pooling 4
    til_pooling4_model=None
    til_pooling4_predict = None
    til_pooling5_model = None
    til_pooling5_predict = None
    if layer in 'pooling4':
        print('pooling4 session')
        #til_pooling4_model = Sequential([layer for layer in model.layers[:15]]) 
        #til_pooling4_model.build((None, 224,224,3))
        til_pooling4_predict = Sequential([layer for layer in model.layers[15:]]) 
        til_pooling4_predict.build((None, 14,14,512))
    return til_pooling4_predict
til_pooling4_predict = ready_model(model_path,'pooling4')
til_pooling4_predict.build((None,14,14,512))

pooling4 session


In [6]:
til_pooling4_predict.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block5_conv1 (Conv2D)        (None, 14, 14, 512)       2359808   
_________________________________________________________________
block5_conv2 (Conv2D)        (None, 14, 14, 512)       2359808   
_________________________________________________________________
block5_conv3 (Conv2D)        (None, 14, 14, 512)       2359808   
_________________________________________________________________
block5_pool (MaxPooling2D)   (None, 7, 7, 512)         0         
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
fc1 (Dense)                  (None, 4096)              102764544 
_________________________________________________________________
fc2 (Dense)                  (None, 4096)              1

In [7]:
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Layer
# elementwise_mul_withfilter_layer
class F4F_layer(tf.keras.layers.Layer):
    def __init__(self,output_dim, **kwargs):
        self.output_dim = output_dim
        super(F4F,self).__init__(**kwargs)
    def build(self, input_shape):
        # 이 레이어에 대해 학습가능한 가중치 변수를 만듭ㄴ디ㅏ.
        self.w = self.add_weight(name='kernel',
                                shape=(input_shape),
                                initializer=tf.contrib.layers.xavier_initializer(),
                                trainable=True)
        super(F4F_layer, self).build(input_shape)
    def call(self,x):
        return self.w * x

In [8]:
conv5_1 = til_pooling4_predict.layers[0]

In [9]:
filter_weights = conv5_1.weights[0]
filter_bias = conv5_1.weights[1]
filter_weights

<tf.Variable 'block5_conv1/kernel:0' shape=(3, 3, 512, 512) dtype=float32, numpy=
array([[[[ 2.70149391e-03, -1.77038671e-03,  8.90818890e-03, ...,
           5.22717601e-03, -1.55181903e-03, -3.73800285e-03],
         [-7.30997371e-03, -3.92698403e-03, -8.54886975e-03, ...,
           2.07138481e-03,  1.28967571e-03, -1.34332357e-02],
         [ 1.61777623e-02,  8.55554268e-03,  6.63516158e-03, ...,
          -2.55896449e-02, -1.94590110e-02,  3.47913810e-05],
         ...,
         [ 8.62450432e-03, -6.13671076e-03, -4.17613750e-03, ...,
           3.92305758e-03,  9.39318212e-04,  4.98578884e-03],
         [ 1.07366564e-02, -9.81058553e-03, -1.00444222e-03, ...,
           7.33137829e-03, -9.91091318e-03,  3.16863763e-04],
         [ 3.30512389e-03,  1.26800162e-03,  3.10195959e-03, ...,
          -5.45442058e-03,  1.22688676e-03,  7.30150146e-03]],

        [[-6.29638648e-03, -1.02038700e-02,  4.13332367e-03, ...,
          -1.22108273e-02, -1.19933486e-02, -8.37498996e-03],
      

In [10]:
F4F = tf.Variable(tf.random.normal(shape=[3,3,512,512]),name="F4F")
#F4F = tf.Variable(name='F4F',shape=[3,3,512,512],initializer=tf.contrib.layers.xaiver_initializer())

In [19]:
filter_weights.shape, F4F.shape

(TensorShape([3, 3, 512, 512]), TensorShape([3, 3, 512, 512]))

In [21]:
# https://www.tensorflow.org/api_docs/python/tf/math/multiply
#multiplied = tf.keras.layers.Multiply()([filter_weights,F4F])
multiplied = tf.math.multiply(filter_weights,F4F)
multiplied.shape

TensorShape([3, 3, 512, 512])

In [29]:
a = filter_weights[0][1][1][511]
b= F4F[0][1][1][511]
print(a,b)
print("Multiply :",a*b)
print("compared :",multiplied[0][1][1][511])

tf.Tensor(-0.015919073, shape=(), dtype=float32) tf.Tensor(0.06517904, shape=(), dtype=float32)
Multiply : tf.Tensor(-0.0010375899, shape=(), dtype=float32)
compared : tf.Tensor(-0.0010375899, shape=(), dtype=float32)


TensorShape([5, 8])

In [31]:
conv5_1.get_config()

{'name': 'block5_conv1',
 'trainable': True,
 'dtype': 'float32',
 'filters': 512,
 'kernel_size': (3, 3),
 'strides': (1, 1),
 'padding': 'same',
 'data_format': 'channels_last',
 'dilation_rate': (1, 1),
 'groups': 1,
 'activation': 'relu',
 'use_bias': True,
 'kernel_initializer': {'class_name': 'GlorotUniform',
  'config': {'seed': None}},
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'kernel_regularizer': None,
 'bias_regularizer': None,
 'activity_regularizer': None,
 'kernel_constraint': None,
 'bias_constraint': None}